In [0]:
new_york_taxis = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferSchema" , "true")\
                .load("/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2019-12.csv.gz")

In [0]:
# new_york_taxis.limit(5).display()

2) Crie uma coluna chamada date_partition que deve ser uma string no formato yyyy-MM-dd extraída da coluna tpep_pickup_datetime.

In [0]:
import pyspark.sql.functions as F

In [0]:
nyTaxis = new_york_taxis.withColumn("date_partition", F.to_date(F.col("tpep_pickup_datetime"), "yyyy-MM-dd")) \
                       
# display(f"DataFrame count: {nyTaxis.count()}")

3) Elimine as linhas com dados inconsistentes que você encontrou na atividade passada.

In [0]:
nyTaxis = nyTaxis.dropna()

# display(f"DataFrame count after dropna: {nyTaxis.count()}")


In [0]:
nyTaxis = nyTaxis.distinct()

# display(f"DataFrame count after distinct: {nyTaxis.count()}")

In [0]:
nyTaxis = nyTaxis.select(("*"), 
                 F.month(("tpep_dropoff_datetime")).alias("month_dropoff"),
                 F.year(("date_partition")).alias("year_dropoff"))

nyTaxis = nyTaxis.filter((F.col("month_dropoff") == 12) & (F.col("year_dropoff") <= 2022))

# display(f"DataFrame count after drop month: {nyTaxis.count()}")


4) Crie uma coluna chama hour que deve ser uma integer extraída da coluna tpep_pickup_datetime.

In [0]:
nyTaxis = nyTaxis.select(("*"), 
                 F.hour(("tpep_pickup_datetime")).alias("hour")) 


5) Crie uma coluna chamada shift que deve ter 4 possíveis valores:
- Madrugada se a corrida tiver começado de 00:00 às 06:59
- Manha se a corrida tiver começado de 07:00 às 11:59
- Tarde se a corrida tiver começado de 12:00 às 17:59
- Noite se a corrida tiver começado de 18:00 às 23:59

In [0]:
from pyspark.sql.functions import when

nyTaxis= nyTaxis.withColumn("shift", \
                when((nyTaxis.hour >= 0) & (nyTaxis.hour < 7), F.lit("Madrugada")) \
                .when((nyTaxis.hour >= 7) & (nyTaxis.hour < 12), F.lit("Manha")) \
                .when((nyTaxis.hour >= 12) & (nyTaxis.hour < 18), F.lit("Tarde")) \
                .otherwise(F.lit("Noite")))

6) Elimine a coluna hour.

In [0]:
nyTaxis = nyTaxis.drop("month_dropoff", "hour", "year_dropoff")

nyTaxis.limit(1).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,date_partition,shift
2,2019-12-01T00:52:28.000+0000,2019-12-01T01:18:46.000+0000,1,9.37,1,N,234,244,1,30.0,0.5,0.5,8.45,0.0,0.3,42.25,2.5,2019-12-01,Madrugada


7) Conte o número de corridas shift.

In [0]:
shift_count = nyTaxis.select("shift") \
                     .groupBy("Shift") \
                     .count() \
                     .withColumnRenamed("count", "trips_shift_count")

display(shift_count)

Shift,trips_shift_count
Madrugada,758265
Tarde,2322888
Manha,1502353
Noite,2260378


8) Conte o número de corridas por dia (1 a 31 de Dezembro). Order o resultado por dia.

In [0]:
df_total_pickup_day = nyTaxis.select(("tpep_pickup_datetime"), 
                            F.dayofmonth(("tpep_pickup_datetime")).alias("tpep_pickup_day"))

df_total_pickup_day = df_total_pickup_day.select("tpep_pickup_day") \
                                       .groupBy("tpep_pickup_day") \
                                       .count() \
                                       .withColumnRenamed("count", "total_pickup_per_day") \
                                       .orderBy("tpep_pickup_day", crescente = True)
display(df_total_pickup_day)

tpep_pickup_day,total_pickup_per_day
1,181825
2,206874
3,247413
4,249042
5,270209
6,273113
7,266752
8,221152
9,223584
10,244623


9) Conte o número de linhas em que a coluna shift é null.

In [0]:
display(nyTaxis.where(F.col("shift").isNull()))


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,date_partition,shift


10) Escreva o novo dataframe em qualquer pasta do databricks de sua preferência, mas o dataframe deve estar particionado por date_partition e no formato parquet.

In [0]:
nyTaxis.count()

Out[232]: 6843884

In [0]:
nyTaxis.write.partitionBy("date_partition").mode("overwrite").parquet("/FileStore/tables/new_nyTaxis.parquet")


11) Leia apenas o dia 2022-12-14 do dataset que você salvou no formato parquet.

In [0]:
df = spark.read.parquet("/FileStore/tables/nyTaxis.parquet/")

df1 = df.select("*") \
        .filter(df.date_partition == "2022-12-14")

df1.display()

In [0]:
# df = spark.read.parquet("/FileStore/tables/nyTaxis.parquet/date_partition=2008-12-31")

df2 = df.select("*") \
        .filter(df.date_partition == "2008-12-31")


In [0]:
df2.display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,shift,date_partition
2,2008-12-31T23:07:29.000+0000,2008-12-31T23:12:25.000+0000,1,0.49,1,N,161,233,2,5.0,1.0,0.5,0.0,0.0,0.3,9.3,2.5,Noite,2008-12-31
2,2008-12-31T23:09:27.000+0000,2008-12-31T23:13:35.000+0000,3,1.25,1,N,143,239,2,6.0,0.0,0.5,0.0,0.0,0.3,9.3,2.5,Noite,2008-12-31
2,2008-12-31T23:31:09.000+0000,2008-12-31T23:33:15.000+0000,1,0.16,1,N,238,238,2,3.5,1.0,0.5,0.0,0.0,0.3,5.3,0.0,Noite,2008-12-31
2,2008-12-31T23:15:28.000+0000,2008-12-31T23:22:07.000+0000,5,0.89,1,N,234,79,2,6.0,0.5,0.5,0.0,0.0,0.3,9.8,2.5,Noite,2008-12-31
2,2008-12-31T18:13:42.000+0000,2008-12-31T18:27:42.000+0000,4,1.12,1,N,50,48,1,9.5,1.0,0.5,3.45,0.0,0.3,17.25,2.5,Noite,2008-12-31
2,2008-12-31T23:20:44.000+0000,2008-12-31T23:24:35.000+0000,1,0.76,1,N,148,232,2,5.0,0.5,0.5,0.0,0.0,0.3,8.8,2.5,Noite,2008-12-31
2,2008-12-31T23:32:31.000+0000,2008-12-31T23:40:05.000+0000,1,0.58,1,N,140,141,2,5.0,1.0,0.5,0.0,0.0,0.3,9.3,2.5,Noite,2008-12-31
2,2008-12-31T22:49:50.000+0000,2008-12-31T23:00:03.000+0000,2,0.04,1,N,148,148,2,7.5,1.0,0.5,0.0,0.0,0.3,11.8,2.5,Noite,2008-12-31
2,2008-12-31T23:14:40.000+0000,2008-12-31T23:20:50.000+0000,4,1.27,1,N,79,137,2,6.5,1.0,0.5,0.0,0.0,0.3,10.8,2.5,Noite,2008-12-31
2,2008-12-31T21:36:45.000+0000,2008-12-31T21:42:54.000+0000,2,0.54,1,N,161,43,1,5.5,1.0,0.5,1.96,0.0,0.3,11.76,2.5,Noite,2008-12-31
